
# Arguments

In [1]:
num_of_epochs = 25
batch_size = 32
dataset_name = 'tomato-dataset'
dataset_path = '../datasets/' + dataset_name
model_save_path = 'model.h5'

checkpoint_path = 'checkpoints.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))

Classes:  ['Tomato___Tomato_mosaic_virus', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Bacterial_spot', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___healthy', 'Tomato___Septoria_leaf_spot', 'Tomato___Two-spotted_spider_mite']
10



# Load model

In [2]:
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
from keras.layers import Input



# Load pretrianed VGG model with FC layers removed
# explicitly deﬁne the input_tensor to be 224×224×3 pixels
base_model = VGG16( weights=None,
                    include_top=False,
                    input_tensor=Input(shape = (input_width,input_height, input_depth)))

plot_save_path = 'diagram-top-removed.png'
# plot_model(base_model, to_file=plot_save_path, show_shapes=True)

Using TensorFlow backend.


# Custom FC layer

In [3]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

head_model = base_model.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(4096,activation='relu')(head_model)
head_model = Dense(4096,activation='relu')(head_model)
# Add a softmaxc layer
head_model = Dense(len(class_names),activation='softmax')(head_model)

# Attach custom head to model

In [4]:
from keras.models import Model


# place the head FC model on top of the base model 
model = Model(inputs=base_model.input, outputs = head_model)

# freeze weights in base model
for layer in base_model.layers:
    layer.trainable = False

plot_save_path = 'diagram-head-attached.png'
# plot_model(model, to_file=plot_save_path, show_shapes=True)

# Compile model

In [5]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001, momentum=0.09)

model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# Since we only train the new fully connected layer, we do a few iterations
print("[INFO] training head...")

[INFO] re-compiling model ...
[INFO] training head...


# Load data

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
print("[INFO] loading images...")
image_paths = list(paths.list_images(dataset_path))

# Initial image preprocessing
aap = AspectAwarePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)



[INFO] loading images...
[INFO]: Processed 500/18160
[INFO]: Processed 1000/18160
[INFO]: Processed 1500/18160
[INFO]: Processed 2000/18160
[INFO]: Processed 2500/18160
[INFO]: Processed 3000/18160
[INFO]: Processed 3500/18160
[INFO]: Processed 4000/18160
[INFO]: Processed 4500/18160
[INFO]: Processed 5000/18160
[INFO]: Processed 5500/18160
[INFO]: Processed 6000/18160
[INFO]: Processed 6500/18160
[INFO]: Processed 7000/18160
[INFO]: Processed 7500/18160
[INFO]: Processed 8000/18160
[INFO]: Processed 8500/18160
[INFO]: Processed 9000/18160
[INFO]: Processed 9500/18160
[INFO]: Processed 10000/18160
[INFO]: Processed 10500/18160
[INFO]: Processed 11000/18160
[INFO]: Processed 11500/18160
[INFO]: Processed 12000/18160
[INFO]: Processed 12500/18160
[INFO]: Processed 13000/18160
[INFO]: Processed 13500/18160
[INFO]: Processed 14000/18160
[INFO]: Processed 14500/18160
[INFO]: Processed 15000/18160
[INFO]: Processed 15500/18160
[INFO]: Processed 16000/18160
[INFO]: Processed 16500/18160
[INFO

# Checkpoints

In [7]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [15]:
#load checkpoints if existing

import os

epochs_done = 0
num_of_epochs = 100

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)
    num_of_epochs = num_of_epochs - epochs_done


# Training monitor

In [16]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'
start_epoch = 0

last_epoch = None
last_improved_epoch = None
load_from_early_epoch = False

if(os.path.exists(values_path) and load_from_early_epoch):
    with open(values_path) as file_content:
        data = json.load(file_content)
        start_epoch = len(data['val_accuracy']) - (last_epoch - last_improved_epoch)
        print('loaded from early epoch')



callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Perform training

In [17]:
# num_of_epochs = 15

model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)



Epoch 1/100
425/425 [==============================] - 209s 492ms/step - loss: 2.2765 - accuracy: 0.2938 - val_loss: 2.2548 - val_accuracy: 0.2985

Epoch 00001: val_loss improved from inf to 2.25479, saving model to checkpoints.hdf5
Epoch 2/100
425/425 [==============================] - 210s 493ms/step - loss: 2.2375 - accuracy: 0.2933 - val_loss: 2.2185 - val_accuracy: 0.2985

Epoch 00002: val_loss improved from 2.25479 to 2.21849, saving model to checkpoints.hdf5
Epoch 3/100
425/425 [==============================] - 204s 479ms/step - loss: 2.2046 - accuracy: 0.2947 - val_loss: 2.1885 - val_accuracy: 0.2985

Epoch 00003: val_loss improved from 2.21849 to 2.18846, saving model to checkpoints.hdf5
Epoch 4/100
425/425 [==============================] - 208s 490ms/step - loss: 2.1792 - accuracy: 0.2933 - val_loss: 2.1649 - val_accuracy: 0.2985

Epoch 00004: val_loss improved from 2.18846 to 2.16492, saving model to checkpoints.hdf5
Epoch 5/100
425/425 [==============================] - 2

# Evaluate after warmup

In [18]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=32)
print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=class_names))

[INFO] evaluating after initialization...
                                        precision    recall  f1-score   support

          Tomato___Tomato_mosaic_virus       0.00      0.00      0.00       532
                 Tomato___Early_blight       0.00      0.00      0.00       239
                  Tomato___Late_blight       0.00      0.00      0.00       470
                    Tomato___Leaf_Mold       0.00      0.00      0.00       238
               Tomato___Bacterial_spot       0.00      0.00      0.00       435
                  Tomato___Target_Spot       0.00      0.00      0.00       362
Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.30      1.00      0.46      1355
                      Tomato___healthy       0.00      0.00      0.00        98
           Tomato___Septoria_leaf_spot       0.00      0.00      0.00       414
      Tomato___Two-spotted_spider_mite       0.00      0.00      0.00       397

                           avg / total       0.09      0.30      0.14      4

/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
